In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
with open('shaytonat 1-3.txt', 'r') as f:
  text = f.read()
bag = list(set(text))
n_bag = len(bag)
print(f'Beliglar soni: {n_bag}')

encode = lambda s: [bag.index(l) for l in s]
decode = lambda ids: "".join([bag[id] for id in ids])

Beliglar soni: 93


In [4]:
n_data = len(text)
val_size = 0.1
n_train = int((1 - val_size) * n_data)
n_val = n_data - n_train

train_data = torch.tensor(encode(text[:n_train]), dtype=torch.int32)
val_data = torch.tensor(encode(text[n_train:]), dtype=torch.int32)
print("O'rgatuvchida: ", n_train)
print("Sinovda: ", n_val)

O'rgatuvchida:  1848219
Sinovda:  205358


In [61]:
batch_size = 4
block_size = 256
n_emb = 32
vocab_size = n_bag

In [5]:
def get_batch(split='train'):
    data = train_data if split == 'train' else val_data
    xb = []
    yb = []
    for i in range(batch_size):
        idx = np.random.randint(0, len(data) - block_size - 1)
        xb.append(data[idx:idx+block_size])
        yb.append(data[idx+1:idx+block_size+1])
    
    xb = torch.stack(xb)
    yb = torch.stack(yb).to(torch.int64)

    return xb, yb

In [ ]:
xb, yb = get_batch()
for b in range(batch_size):
    for c in range(block_size):
        print("Context:", xb[b, :c+1], "Target:", yb[b, c].item())

In [62]:
class BigramLM(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, 
                                                  n_emb)
        self.fc = nn.Linear(n_emb, vocab_size)
    
    def forward(self, xb, yb=None):

        # xb -> (batch_size, block_size) => (4, 8)
        # yb -> (batch_size, block_size) => (4, 8)
        # (batch_size, block_size, n_emb)
        # (4, 8, 32)
        token_emb = self.token_embedding_table(xb)
        # (4, 8, 93)
        logits = self.fc(token_emb)
        
        if yb is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            yb = yb.view(B*T)
            loss = F.cross_entropy(logits, yb)
        else:
            loss = None

        return logits, loss
    
    def generate(self, idx, max_new_token):
        for _ in range(max_new_token):
            # (batch_size, block_size, vocab_size)
            # (1, 8, 93)
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)
            
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [63]:
model = BigramLM()
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

In [64]:
n_steps = 1_000_000
for step in range(n_steps):
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if step % 1000 == 0:
        print(f"Step {step+1}: {loss:.4f}")

Step 1: 4.7315
Step 1001: 2.5747
Step 2001: 2.4736
Step 3001: 2.4962
Step 4001: 2.5640
Step 5001: 2.5076
Step 6001: 2.5378
Step 7001: 2.5091
Step 8001: 2.5681
Step 9001: 2.5342
Step 10001: 2.4651
Step 11001: 2.5051
Step 12001: 2.4897
Step 13001: 2.5188
Step 14001: 2.5003
Step 15001: 2.5557
Step 16001: 2.4764
Step 17001: 2.5909
Step 18001: 2.4757
Step 19001: 2.5070
Step 20001: 2.4815
Step 21001: 2.5283
Step 22001: 2.5349
Step 23001: 2.4788
Step 24001: 2.5269
Step 25001: 2.4810
Step 26001: 2.4577
Step 27001: 2.5570
Step 28001: 2.5132
Step 29001: 2.5005
Step 30001: 2.4844
Step 31001: 2.4825
Step 32001: 2.5147
Step 33001: 2.4517
Step 34001: 2.5306
Step 35001: 2.4912
Step 36001: 2.5263
Step 37001: 2.5307
Step 38001: 2.4754
Step 39001: 2.4713
Step 40001: 2.5591
Step 41001: 2.4564
Step 42001: 2.5453
Step 43001: 2.4818
Step 44001: 2.5044
Step 45001: 2.4279
Step 46001: 2.5144
Step 47001: 2.5354
Step 48001: 2.4683
Step 49001: 2.4864
Step 50001: 2.5325
Step 51001: 2.4973
Step 52001: 2.4738
Step 5

In [67]:
idx = torch.tensor([encode(' ')], dtype=torch.int64, device=device)
gen_idx = model.generate(idx, max_new_token=100)
print(decode(gen_idx[0]))

 қчонг шан кўйда «Себолдадишлиришин б холаригаси. меналла бҳадб ндболоши
— еталарнгухтнибиг бдима Бур


In [7]:
torch.manual_seed(42)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [9]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (1, 2)
        xbow[b, t] = torch.mean(xprev, 0)

In [11]:
x[0], xbow[0]

(tensor([[ 1.9269,  1.4873],
         [ 0.9007, -2.1055],
         [ 0.6784, -1.2345],
         [-0.0431, -1.6047],
         [-0.7521,  1.6487],
         [-0.3925, -1.4036],
         [-0.7279, -0.5594],
         [-0.7688,  0.7624]]),
 tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354],
         [ 0.2272, -0.5388],
         [ 0.1027, -0.3762]]))

In [16]:
a = torch.tril(torch.ones((3, 3)))
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [19]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3, 3)))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [21]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3, 3)))
a = a / torch.sum(a, dim=1, keepdims=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [52]:
torch.manual_seed(42)
wei = torch.tril(torch.ones((T, T)))
wei = wei / torch.sum(wei, dim=1, keepdims=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) => (B, T, C)
torch.allclose(xbow, xbow2)

True

In [33]:
tril = torch.tril(torch.ones((T, T)))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [43]:
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [54]:
torch.manual_seed(42)
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [39]:
xbow3[0]

tensor([[ 1.9269,  1.4873],
        [ 1.4138, -0.3091],
        [ 1.1687, -0.6176],
        [ 0.8657, -0.8644],
        [ 0.5422, -0.3617],
        [ 0.3864, -0.5354],
        [ 0.2272, -0.5388],
        [ 0.1027, -0.3762]])